In [1]:
from os import listdir
from os.path import isfile, join
import re
import fitz

from pprint import pprint
from src.ingestion.cleaner import Cleaner

cleaner = Cleaner()

In [2]:
def extract_text_by_font_size(
        pdf_path, 
        h1_font_size: int=18,
        h2_font_size: int = 15,
        h3_font_size: int= 12
    ):
    doc = fitz.open(pdf_path)
    text_output = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            if block['type'] == 0:  # Text block
                for line in block['lines']:
                    for span in line['spans']:
                        font_size = span['size']
                        text = span['text'].strip()

                        # Use font size to determine hierarchy
                        if font_size >= h1_font_size:
                            text_output.append(f"<h1>{text}</h1>")  # Representing chapter
                        elif font_size >= h2_font_size:
                            text_output.append(f"<h2>{text}</h2>")
                        elif font_size >= h3_font_size:
                            text_output.append(f"<h3>{text}</h3>")
                        else:
                            text_output.append(f"<p>{text}</p>")  # Regular text

    return text_output

In [3]:
pattern = r'(?=<h1>.*?</h1>)'

pdf_path = "Hackapizza Dataset/Misc/Manuale di Cucina.pdf"
html_output = extract_text_by_font_size(pdf_path)
html_content = "\n".join(html_output)

chunks = re.split(pattern, html_content)

chunks = [chunk.strip() for chunk in chunks if chunk.strip()]



In [4]:
for chunk in chunks:
    chunk = cleaner._clean_text(chunk)

    pprint(chunk)

('<h1>Introduzione</h1> Io sono il grande Sirius Cosmo, lo chef stellare per '
 'eccellenza. Se non mi conoscete ancora, preparatevi: il mio nome è sinonimo '
 'di cucina galattica. In questo manuale vi insegnerò tutto quello che serve '
 'per diventare veri cuochi, da Alpha Centauri fino alla Nebulosa del Granchio '
 '(dove, a proposito, non hanno neanche un crostaceo decente). Impareremo '
 'insieme le licenze e le abilità fondamentali per cucinare nello spazio senza '
 "mandare in tilt l'intero sistema di supporto vitale della nave - perché "
 "diciamocelo, un buon brasato non vale una depressurizzazione d'emergenza. Vi "
 'guiderò passo passo tra tecniche di cucina gravitazionali, segreti per '
 'friggere senza far esplodere la cabina e trucchi per ottenere il perfetto '
 'soufflé orbitale. Spoiler: Sì, si può montare una maionese in assenza di '
 "gravità - basta avere il giusto polso e un po' di pazienza, cose che non si "
 'insegnano nei manuali tecnici ma che io, Sirius Cosmo, 

In [5]:
import unicodedata
def clean_corrupted_text(input_text):
    # Normalize text to separate base characters from combining marks
    normalized_text = unicodedata.normalize('NFKD', input_text)
    # Remove combining marks using a regex
    cleaned_text = re.sub(r'[\u0300-\u036f]', '', normalized_text)
    # Remove excessive spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text



In [6]:
for chunk in chunks:
    chunk = cleaner._clean_text(chunk)
    cleaned_text = clean_corrupted_text(chunk)
    pprint(cleaned_text) 

print(len(chunks))


('<h1>Introduzione</h1> Io sono il grande Sirius Cosmo, lo chef stellare per '
 'eccellenza. Se non mi conoscete ancora, preparatevi: il mio nome e sinonimo '
 'di cucina galattica. In questo manuale vi insegnero tutto quello che serve '
 'per diventare veri cuochi, da Alpha Centauri fino alla Nebulosa del Granchio '
 '(dove, a proposito, non hanno neanche un crostaceo decente). Impareremo '
 'insieme le licenze e le abilita fondamentali per cucinare nello spazio senza '
 "mandare in tilt l'intero sistema di supporto vitale della nave - perche "
 "diciamocelo, un buon brasato non vale una depressurizzazione d'emergenza. Vi "
 'guidero passo passo tra tecniche di cucina gravitazionali, segreti per '
 'friggere senza far esplodere la cabina e trucchi per ottenere il perfetto '
 'souffle orbitale. Spoiler: Si, si puo montare una maionese in assenza di '
 "gravita - basta avere il giusto polso e un po' di pazienza, cose che non si "
 'insegnano nei manuali tecnici ma che io, Sirius Cosmo, 

In [7]:
from src.models.manuals import License, LicenseLevel, LicenseCategory

In [8]:
capitolo_1 = cleaner._clean_text(chunks[1])

pattern = r'(?=<h3>.*?</h3>)'
capitolo_1_chunks = re.split(pattern, capitolo_1)

capitolo_1_clean_chunks = [chunk.strip() for chunk in capitolo_1_chunks if chunk.strip()]

category_chunks = []

for chunk in capitolo_1_clean_chunks:
    if chunk.startswith("<h3"):
        pprint(chunk)
        category_chunks.append(chunk)

('<h3>Psionica (P)</h3> Livello 0: Posseduta da tutti se non diversamente '
 'specificato. Tipica degli esseri senzienti. Livello I: lettura pensiero, '
 'telecinesi e teletrasporto di oggetti di massa inferiore a 5 kg, '
 'precognizione e visione del passato fino a 5 minuti Livello II: '
 'manipolazione della probabilità, telecinesi e teletrasporto di oggetti di '
 'massa inferiore a 20 kg, manipolazione delle forze fondamentali '
 "dell'universo Livello III: capacità di donare la coscienza e l'intelletto ad "
 'oggetti, manipolazione della realtà circoscritta a stanze, teletrasporto '
 'senza errore in qualsiasi dimensione temporale, comunione con entità di '
 'altri piani Livello IV: proiezione astrale, riscrittura di realtà '
 'circoscritta a piccole nazioni o asteroidi Livello V: riscrittura di realtà '
 'di intere linee temporali o galassie. Questo livello è equivalente al Grado '
 'di influenza di livello tecnologico III (LTK III)')
('<h3>Temporale (t)</h3> Livello I: effetti te

In [16]:
def parse_category_chunk(chunk: str) -> LicenseCategory:
    # Regex patterns
    header_pattern = r"<h3>(.*?)</h3>"  # Extract the header inside <h3> tags
    level_pattern = r"Livello\s([0-9IVXLn+]+):\s(.*?)(?=Livello\s[0-9IVXLn+]+:|$)"  # Extract levels and descriptions

    # Extract header
    header_match = re.search(header_pattern, chunk)
    header = header_match.group(1) if header_match else "Unknown Header"

    # Extract levels
    levels = []
    for level_match in re.finditer(level_pattern, chunk, re.DOTALL):
        level = level_match.group(1)
        description = level_match.group(2).strip()
        levels.append(LicenseLevel(level=level, level_description=description))

    # Create and return the Pydantic model
    return LicenseCategory(name=header, available_levels=levels)

In [17]:
available_categories = []

for chunk in category_chunks:

    available_categories.append(parse_category_chunk(chunk))

available_categories

[LicenseCategory(name='Psionica (P)', available_levels=[LicenseLevel(level='0', level_description='Posseduta da tutti se non diversamente specificato. Tipica degli esseri senzienti.'), LicenseLevel(level='I', level_description='lettura pensiero, telecinesi e teletrasporto di oggetti di massa inferiore a 5 kg, precognizione e visione del passato fino a 5 minuti'), LicenseLevel(level='II', level_description="manipolazione della probabilità, telecinesi e teletrasporto di oggetti di massa inferiore a 20 kg, manipolazione delle forze fondamentali dell'universo"), LicenseLevel(level='III', level_description="capacità di donare la coscienza e l'intelletto ad oggetti, manipolazione della realtà circoscritta a stanze, teletrasporto senza errore in qualsiasi dimensione temporale, comunione con entità di altri piani"), LicenseLevel(level='IV', level_description='proiezione astrale, riscrittura di realtà circoscritta a piccole nazioni o asteroidi'), LicenseLevel(level='V', level_description='riscr

In [11]:
available_categories[0]

LicenseCategory(name='Psionica (P)', available_levels=[LicenseLevel(level='0', level_description='Posseduta da tutti se non diversamente specificato. Tipica degli esseri senzienti.'), LicenseLevel(level='I', level_description='lettura pensiero, telecinesi e teletrasporto di oggetti di massa inferiore a 5 kg, precognizione e visione del passato fino a 5 minuti'), LicenseLevel(level='II', level_description="manipolazione della probabilità, telecinesi e teletrasporto di oggetti di massa inferiore a 20 kg, manipolazione delle forze fondamentali dell'universo"), LicenseLevel(level='III', level_description="capacità di donare la coscienza e l'intelletto ad oggetti, manipolazione della realtà circoscritta a stanze, teletrasporto senza errore in qualsiasi dimensione temporale, comunione con entità di altri piani"), LicenseLevel(level='IV', level_description='proiezione astrale, riscrittura di realtà circoscritta a piccole nazioni o asteroidi'), LicenseLevel(level='V', level_description='riscri